In [ ]:
import oracledb
import pandas as pd
import getpass
import os

# --- Database Connection Details (Oracle XE) ---
dsn_tns = "localhost:1521/XEPDB1"
db_user = 'system'
db_password = getpass.getpass("Enter Oracle DB password: ") # Prompt for password securely


# Attempt to connect to the database
connection = oracledb.connect(user=db_user, password=db_password, dsn=dsn_tns)
cursor = connection.cursor()
print("Successfully connected to Oracle Database.")

